# **최적화 루프에 대해 알아보기**
모델 훈련은 각 __반복(`epoch`)__ 에서 모델은 출력에 대해 추측하고, 추측에서 __오류(`loss`)__ 를 계산하고 해당 매개 변수에서 도함수 수집후 이를 최적화 한다.  
### **전제 조건 코드**

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork()

### **초 매개 변수 설정**
하이퍼 파리미터는 모델 최적화 프로세스를 제어 할 수 있는 조정 가능한 매개변수이다.   
__훈련을 위해 다음과 같은 하이퍼 파라미터를 정의한다.__
* **에포크** : 데이터 세트를 반복하는 횟수
* **배치 크기** : 각 세대에서 모델에 표시되는 데이터 샘플 수
* **학습률** : 각 배치 / 에포크에서 모델 매개 변수를 업데이트 할 양, 값이 작을수록 학습 속도가 느려지고 크면 학습중에 에측할 수 없는 동작 발생

In [4]:
learning_rate = 1e-3
batch_size= 64
epochs = 5

### **최적화 루프 추가**
최적화 루프를 사용해 모델을 훈련하고 최적화 할 수 있다. 최적화 루프의 각 반복을 __epoch__라고 한다
* **Train Loop** : 훈련 데이터 세트를 반복하고 최적의 매개변수로 수렴
* __검증 / 테스트 루프__ : 모델 성능이 개선되고 있는지 확인하기 위해 테스트 데이터 세트를 반복한다.

### **손실 함수 추가**
__손실함수__ 는 획득한 결과와 목표 값의 유사 정도를 측정하는 것으로 작을 수록 좋다.   
일반적인 손실함수는 회귀 작업을 위한 `nn.MSELoss(평균 제곱 오차)`와 분류를 위한 `nn.NLLLoss(음수 로그 가능성)` 등이 있고 그 외로 모델 출력을 전달하면 정규화 후 예측 오류를 계산하는 `nn.CrossEntropyLoss`가 있고  `nn.LogSoftmax` 등이 있다.

In [5]:
# 손실 기능 초기화
loss_fn = nn.CrossEntropyLoss()

## **최적화 패스**
최적화(optimizer)는 각 학습 단계에서 모델 오류를 줄이기 위해 모델 매개변수를 조정하는 프로세스이다.   
__최적화 알고리즘__ 은 프로세스가 수행되는 __방식__ 을 정의한다.     
옵티마이저는 **SGD 및 ADAM, RMSProp** 등이 있다  

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

**훈련 루프 내 최적화는 3단계이다**   
* `optimizer.zero_grad()` : 모델 매개변수 기울기 재설정
* `loss.backwards()` : 예측 손실을 역전파, 각 매개변수의 손실 기울기 저장
* `optimizer.step()` : 역방향 패스에서 수집된 그라디언트로 매개변수를 조정   

### **완전한 구현**
`train_loop` 최적화 코드로 루프를 정의하고 `test_loop`로 테스트 데이터에 대한 모델 평가

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측 및 손실 계산
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss : {loss:>7f}  [{current:>5d}/{size:>5d}]")
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error : \n Accuracy: {(100 * correct):>0.1f}%, Avg loss {test_loss:>8f} \n")

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n ---------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print('end')

Epoch 1
 ---------------------
loss : 2.281562  [    0/60000]
loss : 2.258693  [ 6400/60000]
loss : 2.209729  [12800/60000]
loss : 2.180859  [19200/60000]
loss : 2.154005  [25600/60000]
loss : 2.076516  [32000/60000]
loss : 2.053840  [38400/60000]
loss : 1.979059  [44800/60000]
loss : 1.954600  [51200/60000]
loss : 1.901312  [57600/60000]
Test Error : 
 Accuracy: 65.1%, Avg loss 0.029650 

Epoch 2
 ---------------------
loss : 1.927979  [    0/60000]
loss : 1.894098  [ 6400/60000]
loss : 1.779630  [12800/60000]
loss : 1.798481  [19200/60000]
loss : 1.690835  [25600/60000]
loss : 1.632533  [32000/60000]
loss : 1.659973  [38400/60000]
loss : 1.556482  [44800/60000]
loss : 1.590102  [51200/60000]
loss : 1.527035  [57600/60000]
Test Error : 
 Accuracy: 65.5%, Avg loss 0.024055 

Epoch 3
 ---------------------
loss : 1.607346  [    0/60000]
loss : 1.580050  [ 6400/60000]
loss : 1.440640  [12800/60000]
loss : 1.503713  [19200/60000]
loss : 1.373647  [25600/60000]
loss : 1.335574  [32000/6000

KeyboardInterrupt: 